## Notebook for creating Simulation 5

See Section 6.5 of Susobhanan+ 2024.

In [14]:
from pint.models import get_model
from pint.simulation import make_fake_toas_uniform
from pint.logging import setup as setup_log
from pint.utils import wavex_setup, dmwavex_setup

from io import StringIO
import numpy as np
import astropy.units as u
from copy import deepcopy

In [15]:
setup_log(level="WARNING")

3

In [27]:
par_sim = """
    PSR           SIM3
    RAJ           05:00:00     1
    DECJ          15:00:00     1
    PEPOCH        55000
    F0            100          1
    F1            -1e-15       1 
    PHOFF         0            1
    DM            15           1
    DM1           0            1
    DM2           0            1  
    TNREDAMP      -13
    TNREDGAM      3.5
    TNREDC        30
    TNDMAMP       -13.5
    TNDMGAM       4
    TNDMC         30
    TZRMJD        55000
    TZRFRQ        1400 
    TZRSITE       gbt
    UNITS         TDB
    EPHEM         DE440
    CLOCK         TT(BIPM2019)
"""

In [23]:
m = get_model(StringIO(par_sim))

In [24]:
ntoas = 2000
toaerrs = np.random.uniform(0.5, 2.0, ntoas) * u.us
freqs = np.linspace(500, 1500, 8) * u.MHz

In [25]:
t = make_fake_toas_uniform(
    startMJD=53001,
    endMJD=57001,
    ntoas=ntoas,
    model=m,
    freq=freqs,
    obs='gbt',
    error=toaerrs,
    add_noise=True,
    add_correlated_noise=True,
    name='fake',
    include_bipm=True,
    include_gps=True,
    multi_freqs_in_epoch=True,
)

In [26]:
# Lines commented out to avoid accidentally rewriting the files. 

m.write_parfile("sim5.par")
t.write_TOA_file("sim5.tim")

In [21]:
m1 = deepcopy(m)
m1.remove_component("PLRedNoise")
m1.remove_component("PLDMNoise")

Tspan = t.get_mjds().max() - t.get_mjds().min()
wavex_setup(m1, Tspan, n_freqs=45, freeze_params=True)
dmwavex_setup(m1, Tspan, n_freqs=45, freeze_params=True)

# Lines commented out to avoid accidentally rewriting the files.
m1.write_parfile("sim5.wx.par")